Linear Regression from Sractch python


In [1]:
import math
import numpy as np
import pandas as pd
import plotly.express as px
import pickle
from google.colab import files
uploaded = files.upload()

Saving test.csv to test.csv


In [2]:
train_data = pd.read_csv('test.csv')
test_data = pd.read_csv('test.csv')
train_data = train_data.dropna()
test_data = test_data.dropna()

In [3]:
train_data.head()
print(type(train_data['x']))
print(type(train_data['x'].values))

<class 'pandas.core.series.Series'>
<class 'numpy.ndarray'>


In [4]:
test_data.head()

,x,y
0,77,79.775152
1,21,23.177279
2,22,25.609262
3,20,17.857388
4,36,41.849864


In [5]:
px.scatter(train_data, x = 'x', y = 'y', template = 'seaborn')

Standaryzacja danych

In [6]:
def standaryzacja(X):
   if isinstance(X, (pd.DataFrame, pd.Series)):
        print(f"Konwersja z {type(X)} na numpy.array")
        X = X.values
   mean = np.mean(X, axis = 0)
   std = np.std(X, axis = 0)
   X = (X - mean)/std
   return X


X_train = standaryzacja(train_data[['x']]) #działa dla wielu feature X_train = standaryzacja(train_data[['x', 'y']])
y_train = train_data['y']
X_test = standaryzacja(test_data[['x']])
y_test = test_data['y']

print(type(X_train))
print(type(X_test))
print('X_train shape: ', X_train.shape)
print('test shape: ', X_test.shape)
# print(X_train.shape[1]) wyskakuje blad tuple index out of range bo mamy wektor 1D (300, )
print(X_test.shape[1])
print('y_train shape: ', y_train.shape)
print('test shape: ', y_test.shape)

Konwersja z <class 'pandas.core.frame.DataFrame'> na numpy.array
Konwersja z <class 'pandas.core.frame.DataFrame'> na numpy.array
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
X_train shape:  (300, 1)
test shape:  (300, 1)
1
y_train shape:  (300,)
test shape:  (300,)


# Linear Regression Model

Linear regression is a fundamental model in machine learning used for predicting a continuous output variable based on input features. The model function for linear regression is represented as:

$$f_{w,b}(x) = wx + b$$

In this equation, $f_{w,b}(x)$ represents the predicted output, $w$ is the weight parameter, $b$ is the bias parameter, and $x$ is the input feature.

## Model Training

To train a linear regression model, we aim to find the best values for the parameters $(w, b)$ that best fit our dataset.

### Forward Pass

The forward pass is a step where we compute the linear regression output for the input data $X$ using the current weights and biases. It's essentially applying our model to the input data.

### Cost Function

The cost function is used to measure how well our model is performing. It quantifies the difference between the predicted values and the actual values in our dataset. The cost function is defined as:

$$J(w,b) = \frac{1}{2m} \sum_{i=1}^{m}(f_{w,b}(x^{(i)}) - y^{(i)})^2$$

Here, $J(w, b)$ is the cost, $m$ is the number of training examples, $x^{(i)}$ is the input data for the $i$-th example, $y^{(i)}$ is the actual output for the $i$-th example, and $w$ and $b$ are the weight and bias parameters, respectively.

### Backward Pass (Gradient Computation)

The backward pass computes the gradients of the cost function with respect to the weights and biases. These gradients are crucial for updating the model parameters during training. The gradient formulas are as follows:

$$
\frac{\partial J(w,b)}{\partial b} = \frac{1}{m} \sum_{i=0}^{m-1} (f_{w,b}(X^{(i)}) - y^{(i)})
$$

$$
\frac{\partial J(w,b)}{\partial w} = \frac{1}{m} \sum_{i=0}^{m-1} (f_{w,b}(X^{(i)}) - y^{(i)})X^{(i)}
$$

## Training Process

The training process involves iteratively updating the weights and biases to minimize the cost function. This is typically done through an optimization algorithm like gradient descent. The update equations for parameters are:

$$w \leftarrow w - \alpha \frac{\partial J}{\partial w}$$

$$b \leftarrow b - \alpha \frac{\partial J}{\partial b}$$

Here, $\alpha$ represents the learning rate, which controls the step size during parameter updates.

By iteratively performing the forward pass, computing the cost, performing the backward pass, and updating the parameters, the model learns to make better predictions and fit the data.


Model implementation


In [8]:
import numpy as np
import pandas as pd
import plotly.express as px
import pickle

class Linear_Regresion:
    """
    Linear Regression Model with Gradient Descent

    Linear regression is a supervised machine learning algorithm used for modeling the relationship
    between a dependent variable (target) and one or more independent variables (features) by fitting
    a linear equation to the observed data.

    This class implements a linear regression model using gradient descent optimization for training.
    It provides methods for model initialization, training, prediction, and model persistence.

    Parameters:
        learning_rate (float): The learning rate used in gradient descent.
        convergence_tol (float, optional): The tolerance for convergence (stopping criterion). Defaults to 1e-6.

    Attributes:
        a (numpy.ndarray): Coefficients (weights) for the linear regression model.
        b (float): Intercept (bias) for the linear regression model.

    Methods:
        initialize_parameters(n_features): Initialize model parameters.
        forward(X): Compute the forward pass of the linear regression model.
        compute_cost(predictions): Compute the mean squared error cost.
        backward(predictions): Compute gradients for model parameters.
        fit(X, y, iterations, plot_cost=True): Fit the linear regression model to training data.
        predict(X): Predict target values for new input data.
        save_model(filename=None): Save the trained model to a file using pickle.
        load_model(filename): Load a trained model from a file using pickle.

    Examples:
        >>> from linear_regression import Linear_Regresion
        >>> model = Linear_Regresion(learning_rate=0.01)
        >>> model.fit(X_train, y_train, iterations=1000)
        >>> predictions = model.predict(X_test)
    """

    def __init__(self, learning_rate, convergence_tol=1e-6):
        self.learning_rate = learning_rate
        self.zbieznosc_tol = convergence_tol
        self.a = None
        self.b = None

    def initialize_parameters(self, n_features):
        """
        Initialize model parameters.

        Parameters:
            n_features (int): The number of features in the input data.
        """
        self.a = np.random.rand(n_features) * 0.01
        self.b = 0

    def forward(self, X):
        """
        Calculate predictions.

        Parameters:
            X (numpy.ndarray): Input data of shape (n_samples, n_features)
                               or a 1D array of shape (n_samples,).

        Returns:
            numpy.ndarray: Predictions of shape (n_samples,).
        """
        if X.ndim == 1:
            X = X.reshape(-1, 1)
        return np.dot(X, self.a) + self.b

    def error(self, predictions):
        """
        Compute the mean squared error cost.

        Parameters:
            predictions (numpy.ndarray): Predictions of shape (n_samples,).

        Returns:
            float: Mean squared error cost.
        """
        n = len(predictions)
        cost = np.sum(((predictions - self.y) ** 2) / (n * 2))
        return cost

    def backward(self, predictions):
        """
        Compute gradients for model parameters.

        Parameters:
            predictions (numpy.ndarray): Predictions of shape (n_samples,).

        Updates:
            derivative_a (numpy.ndarray): Gradient with respect to a.
            derivative_b (float): Gradient with respect to b.
        """
        n = len(predictions)
        error = predictions - self.y  # prediction error
        # gradient for weights (a)
        self.derivative_a = np.dot(error, self.X) / n
        # gradient for bias (b)
        self.derivative_b = np.sum(error) / n

    def fit(self, X, y, iterations, plot_cost=True):
        """
        Fit the linear regression model to the training data.

        Parameters:
            X (numpy.ndarray) or pd.DataFrame/Series: Training input data
                of shape (m, n_features) or (m,).
            y (numpy.ndarray) or pd.DataFrame/Series: Training labels of shape (m,).
            iterations (int): The number of iterations for gradient descent.
            plot_cost (bool, optional): Whether to plot the cost during training. Defaults to True.

        Raises:
            AssertionError: If input data and labels are not NumPy arrays or have mismatched shapes.

        Plots:
            Plotly line chart showing cost vs. iteration (if plot_cost is True).
        """
        if isinstance(X, (pd.DataFrame, pd.Series)):
            print(f"Converting X from {type(X)} to numpy.array")
            X = X.values
        if X.ndim == 1:
            print("Converting from 1D vector (m,) to 2D vector (m,1) to fit a 1D input")
            X = X.reshape(-1, 1)
        if isinstance(y, (pd.DataFrame, pd.Series)):
            print(f"Converting y from {type(y)} to numpy.array")
            y = y.values

        assert isinstance(X, np.ndarray), "X must be a numpy array at this point"
        assert isinstance(y, np.ndarray), "y must be a numpy array"
        assert X.shape[0] == y.shape[0], "X and y must have the same number of rows"
        assert iterations > 0, "iterations must be greater than 0"

        self.X = X
        self.y = y
        self.initialize_parameters(X.shape[1])
        costs = []

        for i in range(iterations):
            predictions = self.forward(X)
            cost = self.error(predictions)
            self.backward(predictions)
            # update weights and bias
            self.a -= self.learning_rate * self.derivative_a
            self.b -= self.learning_rate * self.derivative_b
            costs.append(cost)

            if i % 100 == 0:
                print(f"Iteration: {i}, Cost: {cost}")
            if i > 0 and abs(costs[-1] - costs[-2]) < self.zbieznosc_tol:
                print(f"Convergence reached after {i} iterations")
                break

        if plot_cost:
            fig = px.line(y=costs, title='Cost vs Iteration', template='plotly_dark')
            fig.update_layout(
                title_font_color="#41BEE9",
                xaxis=dict(color="#41BEE9", title="Iterations"),
                yaxis=dict(color="#41BEE9", title="Cost")
            )
            fig.show()

    def predict(self, X):
        """
        Predict target values for new input data.

        Parameters:
            X (numpy.ndarray): Input data of shape (n_samples, n_features)
                               or a 1D array of shape (n_samples,).

        Returns:
            np.ndarray: Predicted values.
        """
        return self.forward(X)

    def save_model(self, filename=None):
        """
        Save the trained model to a file using pickle.

        Parameters:
            filename (str, optional): Path where the model will be saved.
                                      If None, defaults to 'linear_regression_model.pkl'.
        """
        if filename is None:
            filename = 'linear_regression_model.pkl'
        with open(filename, 'wb') as f:
            pickle.dump({'a': self.a, 'b': self.b}, f)

    def load_model(self, filename):
        """
        Load a trained model from a file using pickle.

        Parameters:
            filename (str): Path of the file containing the saved model.
        """
        with open(filename, 'rb') as f:
            params = pickle.load(f)
            self.a = params['a']
            self.b = params['b']



In [9]:
# Usage example
model = Linear_Regresion(0.01)
model.fit(X_train, y_train, iterations=10000)  # handles various array types
y_pred = model.predict(X_test)


Converting y from <class 'pandas.core.series.Series'> to numpy.array
Iteration: 0, Cost: 1732.1368085603087
Iteration: 100, Cost: 236.03935619679038
Iteration: 200, Cost: 35.59270597330102
Iteration: 300, Cost: 8.736928949990093
Iteration: 400, Cost: 5.138800676348992
Iteration: 500, Cost: 4.656724620149305
Iteration: 600, Cost: 4.592136226882874
Iteration: 700, Cost: 4.583482694953435
Iteration: 800, Cost: 4.582323297559157
Convergence reached after 865 iterations


### 1. Mean Squared Error (MSE)

**Formula:**
$$
\text{MSE} = \frac{1}{n} \sum_{i=1}^{n} (y_{\text{true}_i} - y_{\text{pred}_i})^2
$$

**Description:**
- **Mean Squared Error (MSE)** is a widely used metric for evaluating the accuracy of regression models.
- It measures the average squared difference between the predicted values ($y_{\text{pred}}$) and the actual target values ($y_{\text{true}}$).
- The squared differences are averaged across all data points in the dataset.

**Interpretation:**
- A lower MSE indicates a better fit of the model to the data, as it means the model's predictions are closer to the actual values.
- MSE is sensitive to outliers because the squared differences magnify the impact of large errors.

### 2. Root Mean Squared Error (RMSE)

**Formula:**
$$
\text{RMSE} = \sqrt{\text{MSE}}
$$

**Description:**
- **Root Mean Squared Error (RMSE)** is a variant of MSE that provides the square root of the average squared difference between predicted and actual values.
- It is often preferred because it is in the same unit as the target variable, making it more interpretable.

**Interpretation:**
- Like MSE, a lower RMSE indicates a better fit of the model to the data.
- RMSE is also sensitive to outliers due to the square root operation.

### 3. R-squared ($R^2$)

**Formula:**
$$
R^2 = 1 - \frac{\text{SSR}}{\text{SST}}
$$

**Description:**
- **R-squared ($R^2$)**, also known as the coefficient of determination, measures the proportion of the variance in the dependent variable ($y_{\text{true}}$) that is predictable from the independent variable(s) ($y_{\text{pred}}$) in a regression model.
- It ranges from 0 to 1, where 0 indicates that the model does not explain any variance, and 1 indicates a perfect fit.

**Interpretation:**
- A higher $R^2$ value suggests that the model explains a larger proportion of the variance in the target variable.
- However, $R^2$ does not provide information about the goodness of individual predictions or whether the model is overfitting or underfitting.


In [10]:
def Mse(y_true, y_pred):

        """
        Calculate the Mean Squared Error (MSE).

        Args:
            y_true (numpy.ndarray): The true target values.
            y_pred (numpy.ndarray): The predicted target values.

        Returns:
            float: The Mean Squared Error.
        """
        assert(len(y_true) == len(y_pred))
        n = len(y_true)
        return np.mean((y_true - y_pred)**2)
def Rmse(y_true, y_pred):

        """
        Calculate the Root Mean Squared Error (RMSE).

        Args:
            y_true (numpy.ndarray): The true target values.
            y_pred (numpy.ndarray): The predicted target values.

        Returns:
            float: The Mean Squared Error.
        """
        assert(len(y_true) == len(y_pred))
        n = len(y_true)
        return np.sqrt(np.mean((y_true - y_pred)**2))

def R2(y_true, y_pred):
        """
        Calculate the R-squared (R^2) coefficient of determination.

        Args:
            y_true (numpy.ndarray): The true target values.
            y_pred (numpy.ndarray): The predicted target values.

        Returns:
            float: The R-squared (R^2) value.
        """
        assert(len(y_true) == len(y_pred))
        y_mean = np.mean(y_true)
        SSR = np.sum((y_pred - y_test)**2)
        SST = np.sum((y_test - y_mean)**2)
        return 1 - (SSR/SST)


In [11]:
#model metrics
mse = Mse(y_test, y_pred)
print('Mse modelu: ', mse)
rmse = Rmse(y_test, y_pred)
print('Rmse modelu: ', rmse)
r2 = R2(y_test, y_pred)
print('R2 modelu: ', r2)

Mse modelu:  9.164383056644262
Rmse modelu:  3.0272732048238167
R2 modelu:  0.9891202481256606
